### Load data from silver layer to Gold Layer
#### Steps
1. Run the configurations file which has the folder paths defined.
2. Load into gold layer after renaming ColumnNames to Column_Names.

In [0]:

%run "../SetUp/Configuration"

In [0]:

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_replace

In [0]:
# Populate the table names from the silver layer in a list
table_names=[]
for table in dbutils.fs.ls(f'{silver_folder_path}/SalesLT/'):
    table_names.append(table.name.split('/')[0])

for table in table_names:
    df = spark.read \
    .option("header", True)\
    .format('delta')\
    .load(f'{silver_folder_path}/SalesLT/{table}/')
    
    # Rename the columns from ColumnNames to Column_Names
    for col in df.columns:
        new_col_name= "".join("_"+ l if l.isupper()  and not col[i-1].isupper()  else l for i , l in enumerate(col)).lstrip("_")
        df = df.withColumnRenamed(col, new_col_name)

    # Write the modified datafarmes to gold layer    
    output_path = f'{gold_folder_path}/SalesLT/{table}/'
    df.write.format('delta').mode("overwrite").save(output_path)        